In [1]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
%config Completer.use_jedi = False
import SimpleITK as sitk
import nibabel as nib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import logging
from utils_mask_sain import *
from utils import *
from utils_recaler import *


# Global: 3D

In [ ]:
ls_image =sorted(glob.glob('./*_NAT*/*.nii'))

In [89]:
# Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
settings = {}
settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = [2,2]  # en 2D, on travaille dans un plan
settings['resampledPixelSpacing'] = [2, 2, 2]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [80]:
all_dat = pd.DataFrame()
extractor.disableAllFeatures
li_featureClass = ['firstorder', 'shape','glrlm', 'glszm', 'gldm', 'ngtdm']
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
#On ne pourrait pas avoir les 2D...
extractor.enableFeaturesByName(**{"glcm" : ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]})

logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

#OUILLE LES PATIENTS S'APPELLENT ./DATASET et pas en slices???

for imageName in tqdm.tqdm(ls_image):
    maskName = imageName.replace('_NAT','').replace('.nii','_masked.nii')
    classe_name = imageName.split('/')[-2].split('_')[1]
    temps_inj = imageName.split('_')[-1].split('.')[0]

    patient_num = imageName.split('/')[-1].split('_')[0]
    #print(patient_num) 

        
    mask_sitk = sitk.ReadImage(maskName) !=  0  ### ATTENTION C'est important ici on s'interesse au masque et non a l'image masquée
    # print(imageName)
    # print(mask_sitk.GetSize())
    #print(maskName)
    mr_sitk = sitk.ReadImage(imageName)
    #print("nbe voxels :")
    #print(sitk.GetArrayFromImage(mask_sitk).sum())


    featureVector = extractor.execute(mr_sitk, mask_sitk)
    df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
    df_idx['classe_name'] = classe_name
    df_idx['temps_inj'] = temps_inj
    df_idx['patient_num'] = patient_num
    df_idx = pd.DataFrame(df_idx).T
    all_dat = pd.concat([df_idx,all_dat])
    


100%|██████████| 550/550 [06:08<00:00,  1.49it/s]


In [81]:
all_dat.to_csv('./liver_tumors_all_LLB.csv')
all_dat.to_excel('./global_excel.xlsx', index = False)


# Multislice

In [8]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = [2,2]
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0   # ne pas oublier de le changer : fait: ce n'est plus True


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor_sain = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [22]:
time_inj = ["_ART", "_PORT", "_VEIN","_TARD"]
# ls_image_prime = ls_image[(82 + 242):]
ls_image_prime = ls_image
ls_image_no_time = natsorted(list(set([eliminate_temps(x) for x in ls_image_prime])))
ls_image_full_times_surconfiance = [[name +time + ".nii" for time in time_inj] for name in ls_image_no_time]
ls_image_full_time = []
dict_image_full_time = {}
for li_names in ls_image_full_times_surconfiance:
    li_true_names = []
    for i,name in enumerate(li_names):
        if name in ls_image_prime:
            li_true_names.append(name)
    ls_image_full_time.append(li_true_names)
    classe_name = li_names[0].split('/')[-2].split('_')[1]
    patient_num = li_names[0].split('/')[-1].split('_')[0]
    dict_image_full_time[(patient_num, classe_name)] = li_true_names

In [ ]:
time_inj = ["_ART", "_PORT", "_VEIN","_TARD"]

In [23]:
al_dat_slice = pd.DataFrame()
label=extractor.settings.get('label')
label=1
minDims=2
extractor.disableAllFeatures()
extractor_sain.disableAllFeatures()
#utiliser des try et excepts ...
li_featureClass = ['firstorder','shape2D', 'glrlm', 'glszm', 'gldm', 'ngtdm']
li_featureClass_sain = ['firstorder', 'glrlm', 'glszm', 'gldm', 'ngtdm']
li_carac_glcm = ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
for featureClass in li_featureClass_sain:
    extractor_sain.enableFeatureClassByName(featureClass)
extractor.enableFeaturesByName(**{"glcm": li_carac_glcm})
extractor_sain.enableFeaturesByName(**{"glcm": li_carac_glcm})
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

for num, li_image_names in tqdm.tqdm(enumerate(ls_image_full_time)):
    li_mask_names = [image_name.replace('_NAT','').replace('.nii','_masked.nii') for image_name in li_image_names]

    ###### TODo : A REFAIRE
    
    
    classe_name = li_image_names[0].split('/')[-2].split('_')[1]
    temps_inj = li_image_names[0].split('_')[-1].split('.')[0]
    patient_num = li_image_names[0].split('/')[-1].split('_')[0]
    
    li_images = [sitk.ReadImage(image_name) for image_name in li_image_names]
    li_mask = [sitk.ReadImage(mask_name) !=  0 for mask_name in li_mask_names]
    
    li_mask, li_images = equalize_slices(ls_image_full_time, dict_image_full_time,num = num, key = None, show = False)


    #RESTE!!!! A FAIRE
    for slice_num in range(mask_sitk.GetSize()[-1]):
        
        mask_sain_name = mask_sain_name_incomplet + str(slice_num) + '.npy'
        slice_mask = mask_sitk[:,:,slice_num]
        slice_img = mr_sitk[:,:,slice_num]
        
        lsif = sitk.LabelStatisticsImageFilter()
        lsif.Execute(slice_img, slice_mask)
        boundingBox = np.array(lsif.GetBoundingBox(label))
        ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
        #print('ndims ',ndims)
        if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
            # slice_old = slice_img
            slice_mask_sain_np = np.load(mask_sain_name)
            slice_mask_sain = sitk.GetImageFromArray(slice_mask_sain_np)
            slice_mask_sain.CopyInformation(slice_img)
            
            lsif = sitk.LabelStatisticsImageFilter()
            lsif.Execute(slice_img, slice_mask_sain)
            boundingBox = np.array(lsif.GetBoundingBox(label))
            ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
            if (sitk.GetArrayFromImage(slice_mask_sain).sum() > 0) & (ndims >= minDims):
                try:
                    problem = False
                    #print("for extrac",ndims)
                    slice_img = mr_sitk[:,:,slice_num] 
                    feature_vector_sain = extractor_sain.execute(slice_img, slice_mask_sain)
                    #print("extracted")
                    featureVector = extractor.execute(slice_img, slice_mask)
                    featureVector['slice_num'] = slice_num
                    df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                    df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
                    df_idx = pd.concat([df_idx,df_idx_sain])
                    df_idx['classe_name'] = classe_name
                    df_idx['temps_inj'] = temps_inj
                    df_idx['patient_num'] = patient_num
                    # al_dat_slice = al_dat_slice.append(df_idx)
                    df_idx = pd.DataFrame(df_idx).T
                    al_dat_slice = pd.concat([df_idx,al_dat_slice])
                except Exception as e:
                    print(f'erreur pour slice {slice_num} pour image {imageName}')
                    print(e)
                    problem = True
            else:
                problem = True
                print(f'slice {slice_num} non valide sur zone saine pour image {imageName}')
            if problem:
                slice_img = mr_sitk[:,:,slice_num] 
                feature_vector_sain = extractor_sain.execute(slice_img, slice_mask)
                featureVector = extractor.execute(slice_img, slice_mask)
                featureVector['slice_num'] = slice_num
                df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                df_idx_sain = pd.DataFrame(dict([ ("sain_" + k,pd.Series(v)) for k,v in feature_vector_sain.items() ])).iloc[0]
                df_idx_sain[:] = np.nan
                df_idx = pd.concat([df_idx,df_idx_sain])
                df_idx['classe_name'] = classe_name
                df_idx['temps_inj'] = temps_inj
                df_idx['patient_num'] = patient_num
                # al_dat_slice = al_dat_slice.append(df_idx)
                df_idx = pd.DataFrame(df_idx).T
                al_dat_slice = pd.concat([df_idx,al_dat_slice])
     

  3%|▎         | 15/550 [00:24<11:44,  1.32s/it]

slice 85 non valide sur zone saine pour image ./Dataset_CCK_NAT/14_ART.nii


 15%|█▌        | 84/550 [02:40<27:42,  3.57s/it]

slice 49 non valide sur zone saine pour image ./Dataset_CHC_NAT/100_ART.nii
slice 50 non valide sur zone saine pour image ./Dataset_CHC_NAT/100_ART.nii
erreur pour slice 71 pour image ./Dataset_CHC_NAT/100_ART.nii
mask has too few dimensions (number of dimensions 1, minimum required 2)
slice 73 non valide sur zone saine pour image ./Dataset_CHC_NAT/100_ART.nii


 15%|█▌        | 85/550 [02:43<25:56,  3.35s/it]

...

 97%|█████████▋| 535/550 [17:59<00:55,  3.69s/it]

erreur pour slice 73 pour image ./Dataset_Mixtes_NAT/5_TARD.nii
mask has too few dimensions (number of dimensions 1, minimum required 2)


 97%|█████████▋| 536/550 [18:04<00:55,  3.97s/it]

slice 76 non valide sur zone saine pour image ./Dataset_Mixtes_NAT/5_VEIN.nii


100%|██████████| 550/550 [18:20<00:00,  2.00s/it]


In [15]:
print(extractor.enabledFeatures)
print(extractor_sain.enabledFeatures)

{'firstorder': [], 'shape2D': [], 'glrlm': [], 'glszm': [], 'gldm': [], 'ngtdm': [], 'glcm': ['Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointEnergy', 'JointEntropy', 'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']}
{'firstorder': [], 'glrlm': [], 'glszm': [], 'gldm': [], 'ngtdm': [], 'glcm': ['Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointEnergy', 'JointEntropy', 'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']}


In [284]:
a=sitk.GetArrayFromImage(slice_mask)
print(a.sum(axis=1))
print(slice_old.GetDimension())
print(extractor.settings)
label=extractor.settings.get('label')
label=1
lsif = sitk.LabelStatisticsImageFilter()
lsif.Execute(slice_img, slice_mask)
boundingBox = np.array(lsif.GetBoundingBox(label))
print(boundingBox)
ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
print('ndims ',ndims)
a = boundingBox[1::2] - boundingBox[0::2]
print(a)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2
{'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [2, 2], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': True, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'verbose': True}
[225 227 127 129]
ndims  0
[2 2]


In [289]:
extr=extractor.settings
label=extractor.settings.get('label')
label

1

In [24]:
al_dat_slice.to_csv('./liver_tumors_slices_LLB.csv')
al_dat_slice.to_excel('../multislice_excel_with_shape_2D_sain.xlsx', index = False)

In [115]:
print(al_dat_slice)

  diagnostics_Versions_PyRadiomics diagnostics_Versions_Numpy  \
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                        

In [6]:
pd_df  = pd.DataFrame({"a" : [1,2,3], "b" : [4,5,6]})
pd_df[:] = np.nan
print(pd_df)

    a   b
0 NaN NaN
1 NaN NaN
2 NaN NaN
